# Search FORS2 galaxies in KIDS catalogs (FITS)

http://archive.eso.org/scienceportal/home?data_collection=KIDS&publ_date=2019-02-28

https://astroquery.readthedocs.io/en/latest/ukidss/ukidss.html

In [17]:
import matplotlib.pyplot as plt
from astropy.wcs import WCS
from astroquery.ukidss import Ukidss
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
import astropy.coordinates as coordinates
from astropy.coordinates import angular_separation
#(lon1, lat1, lon2, lat2)
from astropy.coordinates import Angle
import astropy.units as u
import h5py
import pandas as pd
import numpy as np
import os
from astropy.io import fits
from astropy.table import Table

In [2]:
def GetColumnHfData(hff,list_of_keys,nameval):
    """
    Extract hff atttribute 
    
    parameters
      hff           : descriptor of h5 file
      list_of_keys : list of exposures
      nameval      : name of the attribute
      
    return
           the array of values in the order of 
    """
    

    all_data = []
    for key in list_of_keys:
        group=hff.get(key)
        val=group.attrs[nameval]
        all_data.append(val)
    return all_data


In [3]:
def ReadFors2h5FileAttributes(hf):
    hf =  h5py.File(input_file_h5, 'r') 
    list_of_keys = list(hf.keys())
    # pick one key    
    key_sel =  list_of_keys[0]
    # pick one group
    group = hf.get(key_sel)  
    #pickup all attribute names
    all_subgroup_keys = []
    for k in group.attrs.keys():
        all_subgroup_keys.append(k)
    # create info
    df_info = pd.DataFrame()
    for key in all_subgroup_keys:
        arr=GetColumnHfData(hf, list_of_keys ,key)
        df_info[key] = arr
    df_info.sort_values(by="num", ascending=True,inplace=True)
    df_info_num = df_info["num"].values
    key_tags = [ f"SPEC{num}" for num in df_info_num ]
    df_info["name"] = key_tags
    
    #'Nsp', 'RT', 'RV', 'Rmag', 'dec', 'eRV', 'lines', 'num', 'ra', 'redshift',
    
    df_info = df_info[['num' ,'name', 'ra', 'dec', 'Rmag','redshift','lines','RT','RV','eRV','Nsp']]
    return df_info
    

## Catalogs

In [4]:
Ukidss.list_catalogs()

['LAS', 'GPS', 'GCS', 'DXS', 'UDS']

In [5]:
Ukidss.list_catalogs(style='long')

['Large Area Survey',
 'Galactic Plane Survey',
 'Galactic Clusters Survey',
 'Deep Extragalactic Survey',
 'Ultra Deep Survey']

In [8]:
path =  "../data/KIDS_Photometry"
filenames_kidscat = os.listdir(path)

In [19]:
all_hdu = []
all_tables = []
for idx,filename  in enumerate(filenames_kidscat):
    fullfilename = os.path.join(path,filename)
    hdu = fits.open(fullfilename)
    all_hdu.append(hdu)
    all_tables.append(Table(hdu[1].data))

In [20]:
all_hdu[0].info()

Filename: ../data/KIDS_Photometry/ADP.2019-02-11T13 02 24.810.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      42   ()      
  1  sourcelist    1 BinTableHDU    437   145258R x 143C   ['J', 'E', 'E', 'J', 'E', 'J', 'J', 'J', 'E', 'E', 'E', 'J', 'E', 'E', 'E', 'E', 'D', 'D', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'J', 'J', 'J', 'J', 'D', 'D', 'E', 'E', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'D', 'D', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'K', 'K', 'K', 'K']   


In [23]:
all_tables[0]

2DPHOT,X_IMAGE,Y_IMAGE,NUMBER,CLASS_STAR,FLAGS,IMAFLAGS_ISO,NIMAFLAGS_ISO,FLUX_RADIUS,KRON_RADIUS,FWHM_IMAGE,ISOAREA_IMAGE,ELLIPTICITY,THETA_IMAGE,MAG_AUTO,MAGERR_AUTO,ALPHA_J2000,DELTA_J2000,FLUX_APER_2,FLUX_APER_4,FLUX_APER_6,FLUX_APER_8,FLUX_APER_10,FLUX_APER_12,FLUX_APER_14,FLUX_APER_16,FLUX_APER_18,FLUX_APER_20,FLUX_APER_22,FLUX_APER_24,FLUX_APER_26,FLUX_APER_28p5,FLUX_APER_30,FLUX_APER_32,FLUX_APER_34,FLUX_APER_36,FLUX_APER_38,FLUX_APER_40,FLUX_APER_42,FLUX_APER_44,FLUX_APER_46,FLUX_APER_48,FLUX_APER_50,FLUX_APER_100,FLUX_APER_200,FLUXERR_APER_2,FLUXERR_APER_4,FLUXERR_APER_6,FLUXERR_APER_8,FLUXERR_APER_10,FLUXERR_APER_12,FLUXERR_APER_14,FLUXERR_APER_16,FLUXERR_APER_18,FLUXERR_APER_20,FLUXERR_APER_22,FLUXERR_APER_24,FLUXERR_APER_26,FLUXERR_APER_28p5,FLUXERR_APER_30,FLUXERR_APER_32,FLUXERR_APER_34,FLUXERR_APER_36,FLUXERR_APER_38,FLUXERR_APER_40,FLUXERR_APER_42,FLUXERR_APER_44,FLUXERR_APER_46,FLUXERR_APER_48,FLUXERR_APER_50,FLUXERR_APER_100,FLUXERR_APER_200,MAG_ISO,MAGERR_ISO,MAG_ISOCOR,MAGERR_ISOCOR,MAG_BEST,MAGERR_BEST,BACKGROUND,THRESHOLD,MU_THRESHOLD,FLUX_MAX,MU_MAX,ISOAREA_WORLD,XMIN_IMAGE,YMIN_IMAGE,XMAX_IMAGE,YMAX_IMAGE,X_WORLD,Y_WORLD,XWIN_IMAGE,YWIN_IMAGE,X2_IMAGE,Y2_IMAGE,XY_IMAGE,X2_WORLD,Y2_WORLD,XY_WORLD,CXX_IMAGE,CYY_IMAGE,CXY_IMAGE,CXX_WORLD,CYY_WORLD,CXY_WORLD,A_IMAGE,B_IMAGE,A_WORLD,B_WORLD,THETA_WORLD,THETA_J2000,ELONGATION,ERRX2_IMAGE,ERRY2_IMAGE,ERRXY_IMAGE,ERRX2_WORLD,ERRY2_WORLD,ERRXY_WORLD,ERRCXX_IMAGE,ERRCYY_IMAGE,ERRCXY_IMAGE,ERRCXX_WORLD,ERRCYY_WORLD,ERRCXY_WORLD,ERRA_IMAGE,ERRB_IMAGE,ERRA_WORLD,ERRB_WORLD,ERRTHETA_IMAGE,ERRTHETA_WORLD,ERRTHETA_J2000,FWHM_WORLD,ISO0,ISO1,ISO2,ISO3,ISO4,ISO5,ISO6,ISO7,SLID,SID,HTM,FLAG
int32,float32,float32,int32,float32,int32,int32,int32,float32,float32,float32,int32,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,float64,float64,float32,float32,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int32,int32,int32,int32,int64,int64,int64,int64
0,12971.978,366.98325,1,0.74624234,2,71,21982,16.706377,3.5,30.967657,20327,0.17988509,-82.62348,14.054873,0.00032719894,13.311838553101927,-28.73393039360626,5.6992278e-09,2.3416197e-08,5.2681486e-08,9.6472384e-08,1.5579491e-07,2.2908603e-07,3.136111e-07,4.0915074e-07,5.130132e-07,6.145272e-07,7.140876e-07,8.1031095e-07,9.0342223e-07,1.0105729e-06,1.0688069e-06,1.1431517e-06,1.2138552e-06,1.2795775e-06,1.3419643e-06,1.3998111e-06,1.4540337e-06,1.504668e-06,1.5526067e-06,1.5985102e-06,1.6432994e-06,2.2949437e-06,2.4631247e-06,4.67983e-11,7.17802e-11,1.0603162e-10,1.3393636e-10,1.6428192e-10,1.9781571e-10,2.2323424e-10,2.5475977e-10,2.7776154e-10,3.0203084e-10,3.2504938e-10,3.4294942e-10,3.609747e-10,3.795313e-10,3.905506e-10,4.0388823e-10,4.1608694e-10,4.2737924e-10,4.3827633e-10,4.482461e-10,4.5735335e-10,4.6605764e-10,4.747949e-10,4.8323057e-10,4.9153753e-10,6.411428e-10,8.658173e-10,14.089065,0.00036437885,14.079053,0.00036875182,14.054873,0.00032719894,7.892016e-13,4.19653e-12,25.587982,2.5280371e-09,17.998192,6.273766e-05,12852,198,13146,565,13.311838553101927,-28.73393039360626,12975.034,367.31403,332.3200479341631,487.512732613337,-20.43380079367604,1.0256833e-06,1.5042576e-06,6.39327e-08,0.00301692

In [24]:
all_tables[1]

2DPHOT,X_IMAGE,Y_IMAGE,NUMBER,CLASS_STAR,FLAGS,IMAFLAGS_ISO,NIMAFLAGS_ISO,FLUX_RADIUS,KRON_RADIUS,FWHM_IMAGE,ISOAREA_IMAGE,ELLIPTICITY,THETA_IMAGE,MAG_AUTO,MAGERR_AUTO,ALPHA_J2000,DELTA_J2000,FLUX_APER_2,FLUX_APER_4,FLUX_APER_6,FLUX_APER_8,FLUX_APER_10,FLUX_APER_12,FLUX_APER_14,FLUX_APER_16,FLUX_APER_18,FLUX_APER_20,FLUX_APER_22,FLUX_APER_24,FLUX_APER_26,FLUX_APER_28p5,FLUX_APER_30,FLUX_APER_32,FLUX_APER_34,FLUX_APER_36,FLUX_APER_38,FLUX_APER_40,FLUX_APER_42,FLUX_APER_44,FLUX_APER_46,FLUX_APER_48,FLUX_APER_50,FLUX_APER_100,FLUX_APER_200,FLUXERR_APER_2,FLUXERR_APER_4,FLUXERR_APER_6,FLUXERR_APER_8,FLUXERR_APER_10,FLUXERR_APER_12,FLUXERR_APER_14,FLUXERR_APER_16,FLUXERR_APER_18,FLUXERR_APER_20,FLUXERR_APER_22,FLUXERR_APER_24,FLUXERR_APER_26,FLUXERR_APER_28p5,FLUXERR_APER_30,FLUXERR_APER_32,FLUXERR_APER_34,FLUXERR_APER_36,FLUXERR_APER_38,FLUXERR_APER_40,FLUXERR_APER_42,FLUXERR_APER_44,FLUXERR_APER_46,FLUXERR_APER_48,FLUXERR_APER_50,FLUXERR_APER_100,FLUXERR_APER_200,MAG_ISO,MAGERR_ISO,MAG_ISOCOR,MAGERR_ISOCOR,MAG_BEST,MAGERR_BEST,BACKGROUND,THRESHOLD,MU_THRESHOLD,FLUX_MAX,MU_MAX,ISOAREA_WORLD,XMIN_IMAGE,YMIN_IMAGE,XMAX_IMAGE,YMAX_IMAGE,X_WORLD,Y_WORLD,XWIN_IMAGE,YWIN_IMAGE,X2_IMAGE,Y2_IMAGE,XY_IMAGE,X2_WORLD,Y2_WORLD,XY_WORLD,CXX_IMAGE,CYY_IMAGE,CXY_IMAGE,CXX_WORLD,CYY_WORLD,CXY_WORLD,A_IMAGE,B_IMAGE,A_WORLD,B_WORLD,THETA_WORLD,THETA_J2000,ELONGATION,ERRX2_IMAGE,ERRY2_IMAGE,ERRXY_IMAGE,ERRX2_WORLD,ERRY2_WORLD,ERRXY_WORLD,ERRCXX_IMAGE,ERRCYY_IMAGE,ERRCXY_IMAGE,ERRCXX_WORLD,ERRCYY_WORLD,ERRCXY_WORLD,ERRA_IMAGE,ERRB_IMAGE,ERRA_WORLD,ERRB_WORLD,ERRTHETA_IMAGE,ERRTHETA_WORLD,ERRTHETA_J2000,FWHM_WORLD,ISO0,ISO1,ISO2,ISO3,ISO4,ISO5,ISO6,ISO7,SLID,SID,HTM,FLAG
int32,float32,float32,int32,float32,int32,int32,int32,float32,float32,float32,int32,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,float64,float64,float32,float32,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int32,int32,int32,int32,int64,int64,int64,int64
0,7365.1724,446.81107,1,0.72392654,3,71,8203,12.831115,3.5,20.21018,7772,0.25279862,-64.27042,13.373684,0.00042204667,13.675079576719337,-28.717905296790736,2.5784194e-08,1.0007197e-07,2.1339194e-07,3.6281017e-07,5.424622e-07,7.4733305e-07,9.775831e-07,1.2249468e-06,1.4752133e-06,1.7127813e-06,1.9209908e-06,2.102267e-06,2.2655734e-06,2.4504916e-06,2.552937e-06,2.6783741e-06,2.8015086e-06,2.9167168e-06,3.02597e-06,3.1287873e-06,3.2200549e-06,3.3071033e-06,3.39044e-06,3.468418e-06,3.5413361e-06,4.44585e-06,4.725451e-06,1.5566365e-10,2.3852492e-10,3.2649763e-10,4.1162349e-10,4.856326e-10,5.6218535e-10,6.340349e-10,7.04967e-10,7.688209e-10,8.2183715e-10,8.6658525e-10,9.0662244e-10,9.454234e-10,9.872461e-10,1.0123158e-09,1.0428282e-09,1.0716171e-09,1.0993122e-09,1.1246976e-09,1.1472591e-09,1.168557e-09,1.1889405e-09,1.2095711e-09,1.2297138e-09,1.2491546e-09,1.7049804e-09,2.8015634e-09,13.530889,0.00048676852,13.521784,0.00049545616,13.521784,0.00049545616,-1.1567431e-12,1.7546754e-11,23.667927,8.422215e-09,16.691584,2.3987655e-05,7222,294,7481,562,13.675079576719337,-28.717905296790736,7366.172,447.4021,176.83541511794328,252.69198205167322,-47.61270354213131,5.455773e-07,7.7994235e-07,1.466868e-07,0.0059571983

In [25]:
all_tables[2]

2DPHOT,X_IMAGE,Y_IMAGE,NUMBER,CLASS_STAR,FLAGS,IMAFLAGS_ISO,NIMAFLAGS_ISO,FLUX_RADIUS,KRON_RADIUS,FWHM_IMAGE,ISOAREA_IMAGE,ELLIPTICITY,THETA_IMAGE,MAG_AUTO,MAGERR_AUTO,ALPHA_J2000,DELTA_J2000,FLUX_APER_2,FLUX_APER_4,FLUX_APER_6,FLUX_APER_8,FLUX_APER_10,FLUX_APER_12,FLUX_APER_14,FLUX_APER_16,FLUX_APER_18,FLUX_APER_20,FLUX_APER_22,FLUX_APER_24,FLUX_APER_26,FLUX_APER_28p5,FLUX_APER_30,FLUX_APER_32,FLUX_APER_34,FLUX_APER_36,FLUX_APER_38,FLUX_APER_40,FLUX_APER_42,FLUX_APER_44,FLUX_APER_46,FLUX_APER_48,FLUX_APER_50,FLUX_APER_100,FLUX_APER_200,FLUXERR_APER_2,FLUXERR_APER_4,FLUXERR_APER_6,FLUXERR_APER_8,FLUXERR_APER_10,FLUXERR_APER_12,FLUXERR_APER_14,FLUXERR_APER_16,FLUXERR_APER_18,FLUXERR_APER_20,FLUXERR_APER_22,FLUXERR_APER_24,FLUXERR_APER_26,FLUXERR_APER_28p5,FLUXERR_APER_30,FLUXERR_APER_32,FLUXERR_APER_34,FLUXERR_APER_36,FLUXERR_APER_38,FLUXERR_APER_40,FLUXERR_APER_42,FLUXERR_APER_44,FLUXERR_APER_46,FLUXERR_APER_48,FLUXERR_APER_50,FLUXERR_APER_100,FLUXERR_APER_200,MAG_ISO,MAGERR_ISO,MAG_ISOCOR,MAGERR_ISOCOR,MAG_BEST,MAGERR_BEST,BACKGROUND,THRESHOLD,MU_THRESHOLD,FLUX_MAX,MU_MAX,ISOAREA_WORLD,XMIN_IMAGE,YMIN_IMAGE,XMAX_IMAGE,YMAX_IMAGE,X_WORLD,Y_WORLD,XWIN_IMAGE,YWIN_IMAGE,X2_IMAGE,Y2_IMAGE,XY_IMAGE,X2_WORLD,Y2_WORLD,XY_WORLD,CXX_IMAGE,CYY_IMAGE,CXY_IMAGE,CXX_WORLD,CYY_WORLD,CXY_WORLD,A_IMAGE,B_IMAGE,A_WORLD,B_WORLD,THETA_WORLD,THETA_J2000,ELONGATION,ERRX2_IMAGE,ERRY2_IMAGE,ERRXY_IMAGE,ERRX2_WORLD,ERRY2_WORLD,ERRXY_WORLD,ERRCXX_IMAGE,ERRCYY_IMAGE,ERRCXY_IMAGE,ERRCXX_WORLD,ERRCYY_WORLD,ERRCXY_WORLD,ERRA_IMAGE,ERRB_IMAGE,ERRA_WORLD,ERRB_WORLD,ERRTHETA_IMAGE,ERRTHETA_WORLD,ERRTHETA_J2000,FWHM_WORLD,ISO0,ISO1,ISO2,ISO3,ISO4,ISO5,ISO6,ISO7,SLID,SID,HTM,FLAG
int32,float32,float32,int32,float32,int32,int32,int32,float32,float32,float32,int32,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,float64,float64,float32,float32,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int32,int32,int32,int32,int64,int64,int64,int64
0,7197.6426,631.593,1,0.83003485,3,70,16777,13.540254,3.5,26.866428,15779,0.13510507,-75.11714,14.038847,0.00035904383,13.675113151771642,-28.717917376035633,8.080049e-09,3.233983e-08,7.163482e-08,1.2521768e-07,1.9145126e-07,2.7396732e-07,3.745633e-07,4.8815235e-07,6.1697455e-07,7.610721e-07,9.0569466e-07,1.0376441e-06,1.155495e-06,1.2879175e-06,1.360226e-06,1.4499137e-06,1.528656e-06,1.5973227e-06,1.6592364e-06,1.7160598e-06,1.7683002e-06,1.8146345e-06,1.8566489e-06,1.8954647e-06,1.931841e-06,2.404146e-06,2.510023e-06,6.833952e-11,1.1036883e-10,1.5622524e-10,1.9448815e-10,2.3519134e-10,2.7796127e-10,3.1983088e-10,3.623092e-10,4.0161355e-10,4.4337198e-10,4.770917e-10,5.055096e-10,5.2979005e-10,5.5755495e-10,5.710722e-10,5.875847e-10,6.0203503e-10,6.147289e-10,6.2588984e-10,6.366844e-10,6.460596e-10,6.5491546e-10,6.6267763e-10,6.702333e-10,6.7731304e-10,7.911701e-10,9.359388e-10,14.04508,0.00037958566,14.039142,0.00038074327,14.039142,0.00038074327,8.8948475e-14,3.6287097e-12,25.523727,2.9604537e-09,17.826754,4.870062e-05,7054,477,7307,755,13.675113151771642,-28.717917376035633,7198.9243,631.6371,228.26595289098168,293.55608874659265,-18.670074253112134,7.043966e-07,9.05945e-07,5.740831e-08,0.0044037625,0.0

NO GALAXY HERE

In [ ]:
assert False

## Read FORS2 catalog

In [6]:
input_file_h5  = '../data/FORS2spectra.hdf5'

In [7]:
hf =  h5py.File(input_file_h5, 'r') 
list_of_keys = list(hf.keys())

In [8]:
df_info = ReadFors2h5FileAttributes(hf)

In [9]:
df_info = df_info[['num', 'ra', 'dec', 'Rmag','redshift']]

In [10]:
df_info

,num,ra,dec,Rmag,redshift
51,2,13.52997,-28.44776,21.18,0.34555
112,3,13.52552,-28.42080,21.45,0.68660
545,9,13.50219,-28.40641,21.60,0.29266
17,13,13.49261,-28.38272,23.14,0.68870
43,19,13.60758,-28.43452,21.70,0.48395
...,...,...,...,...,...
531,734,13.68083,-28.23290,0.00,0.23012
532,735,13.69958,-28.23161,19.28,0.36169
533,736,13.66431,-28.22849,21.64,0.55777
534,737,13.62722,-28.22692,21.40,0.52001


## KIDS catalog

### find central cluster coordinates

In [11]:
target_name_simbad = "BAX 013.5117-28.3994"
target_name_ned = "RX J0054.0-2823"

In [12]:
result_table = Simbad.query_object(target_name_simbad)

In [13]:
result_table

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,SCRIPT_NUMBER_ID
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,int32
BAX 013.5117-28.3994,00 54 02.8,-28 23 58,5,5,--,--,0,D,,1998ApJ...502..558V,1


In [14]:
ra_str = result_table['RA'][0] + ' hours'
dec_str = result_table['DEC'][0] + ' degree'

In [15]:
coord = coordinates.SkyCoord(ra_str,dec_str)
coord

<SkyCoord (ICRS): (ra, dec) in deg
    (13.51166667, -28.39944444)>

In [16]:
coord.ra.deg

13.511666666666665

In [17]:
coord.dec.deg

-28.399444444444445

In [18]:
result_table = Ukidss.query_region(coordinates.SkyCoord(coord.ra.deg,coord.dec.deg,unit=(u.deg, u.deg),
                                           frame='icrs'),radius = 0.25 * u.degree,programme_id="GCS")

In [19]:
result_ptable = result_table.to_pandas()
result_ptable.head()

,sourceID,cuEventID,frameSetID,ra,dec,sigRa,sigDec,epoch,muRa,muDec,...,k_2PA,k_2ErrBits,k_2Deblend,k_2Class,k_2ClassStat,k_2ppErrBits,k_2SeqNum,k_2Xi,k_2Eta,distance


In [20]:
result_ptable.columns

Index(['sourceID', 'cuEventID', 'frameSetID', 'ra', 'dec', 'sigRa', 'sigDec',
       'epoch', 'muRa', 'muDec',
       ...
       'k_2PA', 'k_2ErrBits', 'k_2Deblend', 'k_2Class', 'k_2ClassStat',
       'k_2ppErrBits', 'k_2SeqNum', 'k_2Xi', 'k_2Eta', 'distance'],
      dtype='object', length=229)

In [21]:
N = len(result_table)

In [22]:
all_ra = np.array([ Angle(result_table['RA'][idx] + ' hours').deg for idx in range(N) ])
all_dec = np.array([ Angle(result_table['DEC'][idx] + ' degrees').deg for idx in range(N) ])

In [23]:
catalog_kids = SkyCoord(all_ra*u.degree,all_dec*u.degree)

https://docs.astropy.org/en/stable/coordinates/matchsep.html#astropy-coordinates-matching

In [24]:
all_idx = []  # index of the match
all_d2d = []  # distance in arcsec
all_d3d = []

for index, row in df_info.iterrows():
    c = coordinates.SkyCoord(row['ra']*u.degree, row['dec']*u.degree)
    idx, d2d, d3d = c.match_to_catalog_sky(catalog_simbad)
    all_idx.append(idx)
    all_d2d.append(Angle(d2d[0]).arcsec)
    all_d3d.append(d3d)
    #print(idx,Angle(d2d[0]).arcsec,d3d)
    


NameError: name 'catalog_simbad' is not defined

In [ ]:
plt.hist(all_d2d,bins=50,range=(0,0.2));
plt.xlabel("cross-match angle in arcsec")